# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to Ollama Python Client.
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Chat API : Ollama

Let's start with a Ollama local LLMs

In [2]:
import ollama
print(ollama.list())

# Example of using the ollama library to chat with a model
response = ollama.chat(model='gemma3:1b', messages=[{'role': 'user', 'content': 'Why is the sky blue? in 50 words or less.'}])
print(response.message.content)

models=[Model(model='gemma3:4b', modified_at=datetime.datetime(2025, 6, 18, 0, 35, 9, 741285, tzinfo=TzInfo(+05:30)), digest='a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', size=3338801804, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='4.3B', quantization_level='Q4_K_M')), Model(model='qwen3:1.7b', modified_at=datetime.datetime(2025, 6, 13, 0, 0, 55, 230752, tzinfo=TzInfo(+05:30)), digest='8f68893c685c3ddff2aa3fffce2aa60a30bb2da65ca488b61fff134a4d1730e7', size=1359293444, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='2.0B', quantization_level='Q4_K_M')), Model(model='gemma3:1b', modified_at=datetime.datetime(2025, 5, 10, 11, 10, 25, 297843, tzinfo=TzInfo(+05:30)), digest='8648f39daa8fbf5b18c7b4e6a8fb4990c692751d49917417b8842ca5758e7ffc', size=815319791, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], param

In [1]:
from langchain_ollama import ChatOllama
# Example of using the langchain_ollama library to chat with a model
llm = ChatOllama(model = "gemma3:1b",temperature = 0, format = 'json')
response = llm.invoke("Tell me a joke")
print(response.content.strip())

{

   "Why did the scarecrow win an award? 🍂 Because he was outstanding in his field!":


In [4]:
llm.invoke("What is the capital is France?")

AIMessage(content='{"The capital of France is Paris.":true}', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-06-25T01:59:26.174021Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1282263300, 'load_duration': 113539600, 'prompt_eval_count': 16, 'prompt_eval_duration': 138720000, 'eval_count': 12, 'eval_duration': 586473500, 'model_name': 'gemma3:1b'}, id='run--b4f4da51-447d-40d4-a7f2-8d58cd78d7a2-0', usage_metadata={'input_tokens': 16, 'output_tokens': 12, 'total_tokens': 28})

### Prompt template

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

chat = ChatOllama(model = "gemma3:1b",temperature = 0)

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

In [5]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# Create a ChatPromptTemplate from the template string
prompt_template = ChatPromptTemplate.from_template(template_string)

print("The prompt template:",prompt_template.messages[0].prompt)

print("Input variables of prompt: ", prompt_template.messages[0].prompt.input_variables)

The prompt template: input_variables=['style', 'text'] input_types={} partial_variables={} template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
Input variables of prompt:  ['style', 'text']


In [6]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
# Create a list of messages using the prompt template
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [7]:
print(customer_messages)

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n", additional_kwargs={}, response_metadata={})]


In [8]:
print(customer_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone
. text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [9]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

print(customer_response.content)

Okay, here’s a translation of that text into a more American English style, aiming for a calm and respectful tone:

“Oh my goodness, I’m really upset that my blender lid popped off and splattered the kitchen walls with smoothie. It’s just… a really frustrating situation. I really need your assistance right now, sir.”

**Here’s a breakdown of the changes and why they were made:**

*   **“Arrr” replaced with “Oh my goodness”**: This softens the tone and adds a touch of familiarity.
*   **“fuming” replaced with “really upset”**:  A more natural and less intense phrasing.
*   **“splattered” replaced with “splattered the kitchen walls”**:  More descriptive and avoids a slightly dramatic tone.
*   **“don’t cover the cost of cleaning up” replaced with “It’s just… a really frustrating situation”**:  More polite and acknowledges the problem without dwelling on the details.
*   **“matey” replaced with “sir”**: A standard and respectful greeting.

I hope this is helpful!


In [10]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [11]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [12]:
service_response = chat.invoke(service_messages)
print(service_response.content)

Aye, there, ye landlubber! 

Now, listen close, me hearty. That warranty ain't coverin' the cost o' scrubbin' out yer kitchen, see? It be yer own fault, ye scurvy dog, for forgettin' to lock up yer blender before ye started it. Blast it all, ye be lookin' at it! 

Farewell, then!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [13]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [14]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [15]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [16]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOllama(temperature=0.0, model="gemma3:1b")
response = chat(messages)
print(response.content)

C:\Users\kaila\AppData\Local\Temp\ipykernel_15068\3658663161.py:3: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages)


```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": "slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [17]:
type(response.content) # Str

# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [18]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [19]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [20]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [21]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [22]:
response = chat.invoke(messages)

print(response.content)

```json
{
	"gift": "False",
	"delivery_days": "2",
	"price_value": "slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [23]:
output_dict = output_parser.parse(response.content)

print(output_dict)

{'gift': 'False', 'delivery_days': '2', 'price_value': "slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}


In [24]:
type(output_dict)

output_dict.get('delivery_days')

'2'